<a href="https://colab.research.google.com/github/asthabaj/code-refactor/blob/master/steamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit
!pip install pyngrok==7.1.3
!pip install Pillow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import io

class neunet(nn.Module):
    def __init__(self, num_classes=10):
        super(neunet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        print("Original x shape:", x.shape)
        x = self.pool(torch.relu(self.conv1(x)))
        print("After conv1 and pool:", x.shape)
        x = self.pool(torch.relu(self.conv2(x)))
        print("After conv2 and pool:", x.shape)
        x = x.view(x.size(0), -1)
        print("After view (flattening):", x.shape)
        x = self.fc1(x)
        return x

model = torch.load('/content/drive/MyDrive/imgclassfull.pth', map_location=torch.device('cpu'))
torch.save(model, '/content/drive/MyDrive/new_cpu_model.pth')
model = torch.load('/content/drive/MyDrive/new_cpu_model.pth', map_location=torch.device('cpu'))


model.eval()

class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def predict_image(image):
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = Variable(image)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)

    return class_names[predicted.item()]

st.title("Image Classification")
st.write("Upload an image to classify it.")

uploaded_image = st.file_uploader("Select an image", type="jpg")

if uploaded_image is not None:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image.", use_column_width=True)

    label = predict_image(image)
    st.write(f"Predicted class: {label}")



Overwriting app.py


In [4]:
from pyngrok import ngrok
import os


In [5]:
!ngrok authtoken 2rfrUWWhPmQb9yzVlAKAOJelw1L_2RbdSStE9AdwoKGaGjLk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
ngrok.kill()

In [7]:
public_url = ngrok.connect(8501, 'http').public_url
print(f"Streamlit app is live at {public_url}")


Streamlit app is live at https://49cf-34-127-6-84.ngrok-free.app


In [8]:
!streamlit run --server.port 8501 app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.6.84:8501

2025-01-21 09:07:45.688 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='49cf-34-127-6-84.ngrok-free.app', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/server/browser_websocket_handler.py", line 126, in open
    self._session_id = self._runtime.connect_session(
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/streamlit/runtime/runtime.py", line 

In [9]:
#ngrok.disconnect(public_url)

In [10]:
#!killall ngrok
